# TwitterTracker
Monitor Twitter for mentions of given search terms

In [1]:
import tweepy

### Authentication

In [2]:
credentials = {}
with open('./secrets.txt', 'r') as infile:
    for line in infile.readlines():
        if ":" in line:
            key_val = line.split(': ',1)
            credentials[key_val[0]] = key_val[1].strip()

In [3]:
auth = tweepy.OAuthHandler(credentials['consumer_key'], credentials['consumer_secret'])
auth.set_access_token(credentials['access_token'], credentials['access_token_secret'])
api = tweepy.API(auth)

### Start streaming Tweets

In [4]:
class MyStreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        print(status.text)

In [37]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)

In [81]:
myStream.disconnect()

In [82]:
# myStream.filter(track=['NIKEiD', 'nikeid', 'NikeID'], async=False)
myStream.filter(track=['nike'], async=False)

I just entered to win an #OffWhitePresto by subscribing to @RealSethFowler on YouTube! https://t.co/68NsOOQ91T
RT @inoesasi: Buenas noches mis influencers, ¿ya son embajadores de Adidas o de Nike?
RT @hectordeandam: Buenos días influencers, ya son embajadores y socios de nike o todavía no?
RT @MayCabral4: q cosita lindas los conjuntos adidas y nike, aaaaa mil me compraría 😍❤😍❤😍❤😍❤😍❤
RT @HotNewHipHop: John Elliott x @Nike LeBron Icon slated to release again on 9/1

https://t.co/wrSVGQEhR8
RT @sneaker_wars: 【海外9月8日発売予定】 アクロニウム × ナイキ エアプレスト ミッド 全3色
⇒https://t.co/XxpgAyyFhr
新たなカラーリングがスタンバイ中！
#sneakerwars #スニーカー #NIKE https://t.c…
RT @hectordeandam: Buenos días influencers, ya son embajadores y socios de nike o todavía no?
RT @iiloveroupas: A Nike não brincar em serviço 😍 https://t.co/j9e5sUmBPP
vou lançar um corta vento da Nike rosa, pesadão
I just entered to win an #OffWhitePresto by subscribing to @RealSethFowler on YouTube! https://t.co/PzqdFgHTio
I just entered to win an #OffWhitePrest

KeyboardInterrupt: 

# Analyze the downloaded Tweets

In [12]:
import json
import numpy as np
import pandas as pd

In [152]:
# Load the Tweets
with open('./nike_tweets.json', 'r') as infile:
    raw = infile.readlines()
data = [json.loads(line) for line in raw if line != '\n']
del(data[219]) # Kludge for bad data

# Resave the Tweets in proper JSON format
with open('./nike_tweets_json.json', 'w') as outfile:
    json.dump(data, outfile)
    
# Reload the Tweets as a Pandas dataframe
tweets = pd.read_json('./nike_tweets_json.json')

# Add direct Tweet link to DF
def getTweetUrl(tweet):
    """Returns the URL to a Tweet"""
    root = 'https://twitter.com/{name}/status/{id_}'
    return root.format(name=tweet['user']['screen_name'], id_=tweet['id'])
urls = []
for i in range(len(tweets)):
    urls.append(getTweetUrl(tweets.loc[i])) # Probably not the best way
tweets['url'] = urls

tweets.head(2)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,url
0,NaN,None,2018-08-23 05:07:21,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,0,0,False,{'created_at': 'Thu Aug 16 11:45:41 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...","RT @RING_KNOWS_RING: ブログ更新しました！\n\n"" NIKE KOBE...",2018-08-23 05:07:21.504,False,"{'id': 4505070073, 'id_str': '4505070073', 'na...",https://twitter.com/RING_KNOWS_RING/status/103...
1,NaN,None,2018-08-23 05:54:42,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...","{'media': [{'id': 1032223230294032385, 'id_str...",NaN,0,False,low,...,0,0,False,{'created_at': 'Wed Aug 22 11:14:04 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @koheisblog: 柏レイソル\nFW クリスティアーノ 選手\n『NIKEiD...,2018-08-23 05:54:42.390,False,"{'id': 958656259237949440, 'id_str': '95865625...",https://twitter.com/q3o9IaPXU6vhRQU/status/103...


## Sentiment analysis

In [104]:
import re
from textblob import TextBlob

In [161]:
# https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def get_tweet_sentiment(tweet_text):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet_text))
    score = analysis.sentiment.polarity

    # Return sentiment score and category
    if score > 0:
        return score, 'positive'
    elif score == 0:
        return score, 'neutral'
    else:
        return score, 'negative'

### Add sentiment scores to the Data Frame

In [179]:
pd.Series()

[]

In [184]:
scores = np.zeros(len(tweets))
sentiments = []
for i in range(len(tweets)):
    score, sentiment = get_tweet_sentiment(tweets.loc[i]['text'])
    scores[i] = score
    sentiments.append(sentiment)
    
tweets['sentiment_score'] = scores
tweets['sentiment'] = sentiments

In [186]:
for n in range(len(tweets)):
    tweet = tweets.loc[n]
    
    if tweet['sentiment'] == 'positive':
        print('-'*60)
        print(tweet['text'])
        print(n)
        print(tweet['url'])        

------------------------------------------------------------
J'aime une vidéo @YouTube : "250K GIVEAWAY &amp; LIVE NIKEID/MIADIDAS" à l'adresse https://t.co/nuKQToy4eQ.
4
https://twitter.com/lahssini_othman/status/1032574447193010177
------------------------------------------------------------
Loved my Nike IDs so much I’ve ordered another pair to be designed ready to collect my six star word major medal in… https://t.co/OrXSAEFlgn
5
https://twitter.com/rebeccajade83/status/1032582739915153409
------------------------------------------------------------
@rebeccajade83 @NIKEiD @WMMajors @nycmarathon Good luck. I'm sure you will do great
18
https://twitter.com/HolmsTracy/status/1032646256034828290
------------------------------------------------------------
NIKEiD AIR MAX 2017 NEW BLUE/WHITE [ 918091-991 ] US MEN SZ 14 https://t.co/X16fVBujeW
23
https://twitter.com/RehabCenters_MA/status/1032669067889528833
------------------------------------------------------------
NEW NikeID LeBron XI

------------------------------------------------------------
@snkr_twitr @J23app @NIKEiD You're a brave man for having that white upper lol
327
https://twitter.com/RyShute4/status/1034150217648492544
------------------------------------------------------------
RT @J23app: KD 11 on @NIKEiD 

Design your own -&gt; https://t.co/w241yMe5IR https://t.co/4CpfHYoBGF
328
https://twitter.com/Anidem_hector/status/1034150248585650178
------------------------------------------------------------
@RyShute4 @J23app @NIKEiD You live only once 😂😉
329
https://twitter.com/snkr_twitr/status/1034150624915423233
------------------------------------------------------------
RT @J23app: KD 11 on @NIKEiD 

Design your own -&gt; https://t.co/w241yMe5IR https://t.co/4CpfHYoBGF
334
https://twitter.com/NathanW25296847/status/1034154158239227906
------------------------------------------------------------
@JBfromHouston @NIKEiD That image might have just been a photoshop by the UT social media team, in which case… h

#### Example of a positive Tweet

In [187]:
n = 95
print(tweets.loc[n]['text'])
print(tweets.loc[n]['url'])

NIKEiD customs arrived! Best ones yet 😊 Bball # 10 &amp; born in '85 #LoveThese https://t.co/IlGKZPgwnG
https://twitter.com/dino/status/1032986865605865472


#### Example of a negative Tweet

In [188]:
# Example of a negative Tweet
n = 203
print(tweets.loc[n]['text'])
print(tweets.loc[n]['url'])

dunks not on nikeid no more 😤😤😤😤
https://twitter.com/jawny/status/1033367723349299200


## Visualize Tweet sentiments over time

In [193]:
import matplotlib

In [191]:
tweets['sentiment_score'].plot()

ModuleNotFoundError: No module named 'matplotlib'